# Paleo AI Viewer

This notebook loads Holocene paleotemperature data, applies an ensemble of trained neural networks, and generates an interactive scatter plot comparing predicted and true temperature values.

Link the notebook to Colab (Do not use if directly oppened from Colab)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ArturStachnik/Paleo_AI_viewer/blob/main/Paleo_AI_Viewer.ipynb)

Load Ensemble and Data to calculate residuals

In [8]:
#Install dependencies
!pip install -q torch==2.* plotly pandas scikit-learn joblib

#Load from GitHub
!git clone https://github.com/ArturStachnik/Paleo_AI_viewer.git repo

#Imports
import os, glob, torch, joblib
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split

#File paths
MODELS_DIR = "repo/models"
CSV_PATH   = "repo/Holocene_T_clean.csv"

#Define Model and Utilities
class ProbNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(128, 64), nn.ReLU(),
        )
        self.loc     = nn.Linear(64, 1)
        self.log_var = nn.Linear(64, 1)
    def forward(self, x):
        h = self.net(x)
        return self.loc(h), self.log_var(h)

def load_ensemble(model_dir, device="cpu"):
    scaler = joblib.load(os.path.join(model_dir, "scaler.pkl"))
    input_dim = scaler.scale_.shape[0] + 3
    models = []
    for path in sorted(glob.glob(os.path.join(model_dir, "model_*.pt"))):
        m = ProbNet(input_dim).to(device)
        m.load_state_dict(torch.load(path, map_location=device))
        m.eval()
        models.append(m)
    return models, scaler

def predict_mean(models, X):
    X_t = torch.tensor(X, dtype=torch.float32)
    with torch.no_grad():
        mus = [m(X_t)[0].cpu().numpy().flatten() for m in models]
    return np.mean(mus, axis=0)

#Load and prepare data
RAW_DF = pd.read_csv(CSV_PATH)
seasons    = ["annual", "warm", "cold"]
season_ohe = pd.get_dummies(RAW_DF["season"])[seasons].values
cont       = RAW_DF[["lon", "lat", "elev", "age", "resolution"]].values
y          = RAW_DF["temperature"].values

Xc_tr, Xc_te, y_tr, y_te, s_tr, s_te, idx_tr, idx_te = train_test_split(
    cont, y, season_ohe, RAW_DF.index,
    test_size=0.2, random_state=0, shuffle=True
)

device     = "cuda" if torch.cuda.is_available() else "cpu"
models, sc = load_ensemble(MODELS_DIR, device)
X_test     = np.hstack([sc.transform(Xc_te), s_te])
mu_pred    = predict_mean(models, X_test)

#Create DataFrame
plot_df = pd.DataFrame({
    "True":       y_te,
    "Predicted":  mu_pred,
    "season":     RAW_DF.loc[idx_te, "season"].values,
    "proxy":      RAW_DF.loc[idx_te, "proxy"].values,
    "archive":    RAW_DF.loc[idx_te, "archive"].values,
    "dataSetName":RAW_DF.loc[idx_te, "dataSetName"].values,
})

Cloning into 'repo'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 869.77 KiB | 3.70 MiB/s, done.


Residuals Interactive Graph: The plot includes a dropdown menu to color-code the points by season, proxy, or archive type.

In [12]:
import plotly.express as px
import plotly.graph_objects as go

#Create figures with different colorings
fig_blank   = px.scatter(plot_df, x="True", y="Predicted", hover_data=["proxy","season","archive","dataSetName"])
fig_season  = px.scatter(plot_df, x="True", y="Predicted", color="season",  hover_data=["proxy","archive","dataSetName"])
fig_proxy   = px.scatter(plot_df, x="True", y="Predicted", color="proxy",   hover_data=["season","archive","dataSetName"])
fig_archive = px.scatter(plot_df, x="True", y="Predicted", color="archive", hover_data=["proxy","season","dataSetName"])

data = fig_blank.data + fig_season.data + fig_proxy.data + fig_archive.data
fig  = go.Figure(data=data)
nb, ns, np_, na = map(len, [fig_blank.data, fig_season.data, fig_proxy.data, fig_archive.data])

#Add identity line
fig.add_trace(go.Scatter(
    x=[plot_df["True"].min(), plot_df["True"].max()],
    y=[plot_df["True"].min(), plot_df["True"].max()],
    mode="lines",
    line=dict(color="gray", dash="dash"),
    name="Ideal (y = x)"
))

#Layout with dropdown
fig.update_layout(
    title="True vs Predicted (Blank)",
    yaxis_scaleanchor="x", height=1000, width=2000, #Chage this to modify the graph dimensions
    updatemenus=[dict(
        buttons=[
            dict(method="update", label="Blank", args=[{"visible":[True]*nb + [False]*(ns+np_+na) + [True]}, {"title": "Blank"}]),
            dict(method="update", label="By Season", args=[{"visible":[False]*nb + [True]*ns + [False]*(np_+na) + [True]}, {"title": "By Season"}]),
            dict(method="update", label="By Proxy", args=[{"visible":[False]*(nb+ns) + [True]*np_ + [False]*na + [True]}, {"title": "By Proxy"}]),
            dict(method="update", label="By Archive", args=[{"visible":[False]*(nb+ns+np_) + [True]*na + [True]}, {"title": "By Archive"}]),
        ],
        direction="down", showactive=True, x=0, y=1.15
    )]
)

fig.update_traces(marker=dict(size=6, opacity=0.7))
fig.show()
